# HMM Lotka-Volterra Markov Jump Process 

This notebook will demonstrate how to use SequentialMonteCarlo.jl for a Hidden Markov Model with Lotka-Volterra Markov Jump Process dynamics.

First we load the necessary packages.

In [ ]:
using SequentialMonteCarlo
using Distributions
using Catalyst
using JumpProcesses
using Statistics
using Plots
using StatsPlots

Next load in the observed data

In [ ]:
ts = 1.0:1.0:50.0
n = 50
y = [25 54 70 106 155 187 153 104 54 23 15 6 4 7 11 18 26 22 37 43 55 114 190 203 201 97 46 21 29 6 10 13 10 13 18 20 23 40 66 81 119 157 196 227 154 61 24 20 10 11;
    59 45 50 48 89 113 203 378 422 350 288 228 150 124 118 101 67 54 63 48 49 41 82 130 270 435 450 366 280 263 213 159 100 89 71 50 33 45 45 23 45 59 110 223 479 435 406 322 246 201]

SequentialMonteCarlo.jl is a flexible package for designing SMC algorithms. It has three major components:
- Particle structure
- Mutation function (includes initial distribution)
- Potential function (log-scale)

The first step is to define the type of particle the algorithm will manipulate.

In [ ]:
mutable struct LVParticle
    prey::Int64
    pred::Int64
    LVParticle() = new() # initialise empty
end

In order to define the mutation function, we need to define how the Markov Process Jump evolves. 

In julia we can use JumpProcesses.jl to simulate integer-valued Markov Jump Processes. One way to do this is with the Catalyst.jl reaction network syntax.

The Lotka-Volterra model has a chemical reaction network 
\begin{align*}
X_1 & \overset{c_1}{\rightarrow} 2X_1\\
X_1 + X_2 & \overset{c_2}{\rightarrow} 2X_2\\
X_2 & \overset{c_3}{\rightarrow} \emptyset
\end{align*}

which can be written as

In [ ]:
lveqns = @reaction_network begin
    c1, prey --> 2*prey         # prey reproduction
    c2, prey + pred --> 2*pred  # prey death, pred reproduction
    c3, pred --> 0              # pred death
end

rates = (:c1 => 0.5, :c2 => 0.0025, :c3 => 0.3)
initial = [71, 79] # latent at time 0.0

Using the above specification, we can define a mutation kernel for SequentialMonteCarlo.jl to use.

In [ ]:
# mutation kernel
function M!(newParticle::LVParticle, rng, t::Int64, oldParticle::LVParticle, ::Nothing)
    if t == 1
        u0 = initial
        tspan = (0.0, ts[1])
    else
        u0 = [oldParticle.prey, oldParticle.pred]
        tspan = (ts[t-1], ts[t])
    end
    
    dprobt = DiscreteProblem(lveqns, u0, tspan, rates)
    jprobt = JumpProblem(lveqns, dprobt, Direct(), rng = rng)
    res = solve(jprobt, SSAStepper())

    newParticle.prey = res(ts[t])[1]
    newParticle.pred = res(ts[t])[2]
end

Last we have the potential function (likelihood) for which we assume 
\begin{align}
Y_1 &\sim \text{Poisson}(0.5X_1)\\
Y_2 &\sim \text{Poisson}(0.8X_2)
\end{align}
which defines our log-likelihood

In [ ]:
function lG(t::Int64, particle::LVParticle, ::Nothing)
    preydist = Poisson(0.5*particle.prey)
    preddist = Poisson(0.8*particle.pred)
    return logpdf(preydist, y[1,t]) + logpdf(preddist, y[2,t])
end

To run the SMC algorithm we need to specify the number of particles $N$, number of threads, the resampling threshold in terms of the relative ESS, and whether to save all the output (instead of just the terminal particles).

In [ ]:
N = 2^10
threads = 1
κ = 0.5 # relative ESS threshold
saveall = true

The complete SMC model can be specified by

In [ ]:
model = SMCModel(M!, lG, n, LVParticle, Nothing)

and we make a location to save the output from running the algorithm

In [ ]:
smcio = SMCIO{model.particle, model.pScratch}(N, n, threads, saveall, κ)
smcio.N

Then we can run the model using

In [ ]:
smc!(model, smcio) # overwrites smcio

We can plot some path realisations (predictive) as follows

In [ ]:
# load true latent states
x = readdlm("prey_pred_true_latent.csv", ',', Int64)

plot(y[1,:], label="obs", title = "Prey estimated latent states vs observations")
pids = sample(1:smcio.N, 100)
for i in pids
    anc = getindex.(smcio.allEves, i)
    u = getindex.(smcio.allZetas, reverse(anc))
    plot!(getfield.(u, :prey), color = "grey", alpha = 0.5, label = "")
end
plot!(x[1,:], label="latent", color = "red")

In [ ]:
plot(y[2,:], label="obs", title = "Pred. estimated latent states vs observations")
pids = sample(1:smcio.N, 100)
for i in pids
    anc = getindex.(smcio.allEves, i)
    u = getindex.(smcio.allZetas, reverse(anc))
    plot!(getfield.(u, :pred), color = "grey", alpha = 0.5, label = "")
end
plot!(x[2,:], label="latent", color = "red")

## Exercises

1. Plot the $t=5$ marginal predictive and updated distributions of the particles and compare. Hint: consider using `density(..., weights = Weights(...))`.

2. Change the adaptive resampling threshold $\kappa$ and rerun the algorithm. Does your chosen value of $\kappa$ result in a better particle filter? Justify why or why not with summaries from the particle filter.

3. Change the parameters of the model $c_1,c_2,c_3$ slightly and rerun the particle filter. Can you determine how sensitive the particle filter is to the choice of parameters? Does it perform better or worse with your choice? What might be some reasons for this?
